In [3]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import matplotlib.animation as animation

In [4]:
L = 250
N = 100
x = np.arange(-int(L/2), int(L/2), 1)
y = np.arange(-int(L/2), int(L/2), 1)
delta_theta = np.zeros(N)
v = np.ones(N)*0.3

##x represents coordinates, v has the components of the velocity (probably superfluous, psi is the orientation, omega is the 
##angular velocity and r is the radius.
particles_x = np.zeros([N, 2])
particles_v = np.zeros([N, 2])
particles_omega = np.zeros(N)
particles_psi = np.zeros(N) + np.random.uniform(-np.pi/4, np.pi/4)
particles_r = np.zeros(N)

In [9]:
for i in range (int(N/10)):
    particles_x[10*i:(10*(i+1)), 0] = np.arange(0, 10, 1) + np.random.uniform(-0.1, 0.1, 10)
    particles_x[int(N*i/10): int(N*(i+1)/10), 1] = np.arange(0, int(N/10), 1) + np.random.uniform(-0.1, 0.1, int(N/10))
    
particles_r[:] = np.random.normal(1, 0.1, N)
particles_v[:, 0] = v*np.cos(particles_psi)
particles_v[:, 0] = v*np.sin(particles_psi)

In [4]:
def calcDTheta (N, particles_x, selectMatrix):
    theta_out = np.zeros(N)
    theta_in = np.zeros(N)
    d_theta = np.zeros(N)
    
    particles_dist = particles_x[None, :, :] - particles_x[:, None, :]
    
    ##we need to avoid dividing by zero
    np.fill_diagonal(particles_dist[:, :, 0], 100)
    np.fill_diagonal(particles_dist[:, :, 1], 100)
    
    angles = selectMatrix*np.arctan2(particles_dist[:, :, 1], particles_dist[:, :, 0])
    
    #we correct the diagonal elements that we arbitrarily changed two lines above
    np.fill_diagonal(angles[:, :], 0)
    
    for i in range (N):
        M = max(angles[i, :])
        m = min(angles[i, :])
        if M > 0 and m < 0:
            if M - m < np.pi:
                theta_in[i] = (M-m)/2
                theta_out[:] = 2*np.pi - 2*theta_in
                d_theta = particles_psi - theta_in
        elif (M > 0 and m > 0) or (M < 0 and m < 0):
            theta_in[i] = (M-m)/2
            theta_out[:] = 2*np.pi - 2*theta_in
            d_theta = particles_psi - theta_in
    
    return theta_in, theta_out, d_theta

def forceCalc(v, theta_out, d, particles_psi, particles_omega, selectMatrix):
    psi_diff = (particles_psi[None, :] - particles_psi[:, None])*selectMatrix
    
    F = 0.05 + (theta_out[:] - np.pi)*0.3*np.piecewise(theta_out[:] -np.pi, [theta_out[:] -np.pi > 0], [0, 1]) - 1*np.sum(d[:], axis =1)
    F_x = F*np.cos(particles_psi[:])
    F_y = F*np.sin(particles_psi[:])
    
    T = 0.06*np.piecewise(theta_out[:] -np.pi, [theta_out[:] -np.pi > 0], [0, 1]) + 0.03*np.random.uniform(-1,1, N) + 0.5*np.sum(psi_diff, axis = 1)[:]
    
    return F_x, F_y, F, T
    
def update(N, L, v, particles_x, particles_r, particles_psi, particles_omega):
    ##distances calculation
    d = np.linalg.norm(particles_x[None, :, :] - particles_x[:, None, :], axis = 2)
    
    selectMatrix = np.copy(d)
    selectMatrix[selectMatrix > 2.7] = 0
    selectMatrix[selectMatrix > 0] = 1 ##the selection matrix is used to only take into account the neighbours with d < 2.7
    
    ##OPTIMIZATION NEEDED
    for i in range (0, N):
        for j in range (0, N):
            d[i, j] -= (particles_r[i] + particles_r[j])
    d[d>0] = 0
    d = - d
    
    ##calculations
    theta_in, theta_out, d_theta= calcDTheta(N, particles_x, selectMatrix)
    
    F_x, F_y, v, T = forceCalc(v, theta_out, d, particles_psi, particles_omega, selectMatrix)
    
    particles_x[:, 0] = np.mod(particles_x[:, 0] + F_x[:], L)
    particles_x[:, 1] = np.mod(particles_x[:, 1] + F_y[:], L)
    particles_omega[:] = - (particles_psi[:] - T*1) + particles_omega[:]
    particles_psi[:] = particles_psi[:] + particles_omega[:]
    
    return v, F_x, F_y, particles_x, particles_psi, particles_omega

In [5]:
plt.scatter(particles_x[:, 0], particles_x[:, 1], s = np.pi*particles_r[:]**2)
plt.show()

In [6]:
v, F_x, F_yparticles_x, particles_psi, particles_omega = update(N, L, v, particles_x, particles_r, particles_psi, particles_omega)

ValueError: too many values to unpack (expected 5)

In [ ]:
sizes = np.pi *particles_r
for i in range (10**6):
    v, F_x, F_y, particles_x, particles_psi, particles_omega = update(N, L, v, particles_x, particles_r, particles_psi, particles_omega)
    print(i)
    if i > (10**6 - 50):
        plt.scatter(particles_x[:, 0], particles_x[:, 1], s= 100*sizes)
        plt.axis([0, L, 0, L])
        plt.show()
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27